In [155]:
'''
Exercise2.2: implement eliminate()

Now, let's finish the code for the function eliminate(), which will take as input a puzzle in dictionary form. 
The function will iterate over all the boxes in the puzzle that only have one value assigned to them, 
and it will remove this value from every one of its peers.

'''
#1. utils.py ----------------------------
#1.1 define rows: 
rows = 'ABCDEFGHI'

#1.2 define cols:
cols = '123456789'

#1.3 cross(a,b) helper function to create boxes, row_units, column_units, square_units, unitlist
def cross(a, b):
    return [s+t for s in a for t in b]

#1.4 create boxes
boxes = cross(rows, cols)

#1.5 create row_units
row_units = [cross(r, cols) for r in rows]

#1.6 create column_units
column_units = [cross(rows, c) for c in cols]

#1.7 create square_units for 9x9 squares
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

#1.8 create unitlist for all units
unitlist = row_units + column_units + square_units

#1.9 create peers of a unit from all units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

#1.10 display function receiving "values" as a dictionary and display a 9x9 suduku board
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    
    sudoku_list = list(grid)
    sudoku_dict = dict()
    i = 0
    for box in boxes:
        sudoku_dict[box] = '123456789' if sudoku_list[i]=='.' else sudoku_list[i]
        i = i+1
    return sudoku_dict

#2. function.py ----------------------------
# 2.1 implement eliminate(values)
# from utils import *
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    from copy import deepcopy
    values_new = deepcopy(values)
    
    for key, value in values.items():
        value = str(value)
        if len(value) == 0:
            return False
        if len(value) == 1:
            for a in units[key][0]:
                if a!=key:
                    value_temp = list(values_new[a])
                    if value in value_temp:
                        value_temp.remove(value)
                        values_new[a] = ''.join(value_temp)
            for b in units[key][1]:
                if b!=key:
                    value_temp = list(values_new[b])
                    if value in value_temp:
                        value_temp.remove(value)
                        values_new[b] = ''.join(value_temp)
            for c in units[key][2]:
                if c!=key:
                    value_temp = list(values_new[c])
                    if value in value_temp:
                        value_temp.remove(value)
                        values_new[c] = ''.join(value_temp)
    return values_new

#2. function.py ----------------------------
# 2.1 implement only_choice(values)
# from utils import *
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    from copy import deepcopy
    values_new = deepcopy(values)
    for key, value in values.items():
        if len(value) > 1:
            value_temp = set(value)
            for a in units[key][0]:
                if key != a:
                    value_temp.difference_update(set(values[a]))
            if len(value_temp) == 1:
                values_new[key] = ''.join(value_temp)
                continue
            value_temp = set(value)
            for b in units[key][1]:
                if key != b:
                    value_temp.difference_update(set(values[b]))
            if len(value_temp) == 1:
                values_new[key] = ''.join(value_temp)
                continue
            value_temp = set(value)
            for c in units[key][2]:
                if key != c:
                    value_temp.difference_update(set(values[c]))
            if len(value_temp) == 1:
                values_new[key] = ''.join(value_temp)
                continue
    return values_new

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    from copy import deepcopy
    y_prev = deepcopy(values)
    for i in range(500):
        z = eliminate(y_prev)
        if z == False:
            return False
        y = only_choice(z)
        if y_prev==y:
            return y
        y_prev = deepcopy(y)
    return y_prev
    

#2. function.py ----------------------------
# 2.1 implement search() using Depth First Search Algorithm
#from utils import *
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    # Search and Choose one of the unfilled squares with the fewest possibilities
    # Now use recursion to solve each one of the resulting sudokus, 
    # and if one returns a value (not False), return that answer!
    def do_search(values, box, boxes):
        possible_value = list(values[boxes[box]])
        for value in possible_value:
            values[boxes[box]] = value
            values_temp = reduce_puzzle(values)
            if values_temp == False:
                continue
            if all(len(values_temp[b])==1 for b in boxes):
                return values_temp
            from copy import deepcopy
            temps = do_search(deepcopy(values_temp), box + 1, boxes)
            if temps:
                return temps
        return False
                
    
    values = reduce_puzzle(values)
    temp = dict()
    for key, value in values.items():
        if len(value) > 1:
            if len(value) in list(temp.keys()):
                temp[len(value)].append(key)
            else:
                temp[len(value)] = list()
                temp[len(value)].append(key)
    boxes = list()
    for key in sorted(list(temp.keys())):
        for value in temp[key]:
            boxes.append(value)
    return do_search(values, 0, boxes)

#3. Test utils.py ----------------------------  
grid_easy = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid_hard = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid_hard)
print("The original Sudoku board is **********************************************")
display(values)

#4. Test function.py ----------------------------  
new_values = search(values)
print("\n")
print("After applying Depth First Search Algorithm *****************")
display(new_values)

The original Sudoku board is **********************************************
    4     123456789 123456789 |123456789 123456789 123456789 |    8     123456789     5     
123456789     3     123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |    7     123456789 123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     2     123456789 |123456789 123456789 123456789 |123456789     6     123456789 
123456789 123456789 123456789 |123456789     8     123456789 |    4     123456789 123456789 
123456789 123456789 123456789 |123456789     1     123456789 |123456789 123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789 123456789 |    6     123456789     3     |123456789     7     123456789 
    5     123456789 123456789 |    2     123456789 123456789 |123456789 123456789 12345